#Домашнє завдання до тем apply(), groupby(), pivot_table()

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [2]:
import pandas as pd

data_path = '../dataset/supermarket_sales.csv'
df = pd.read_csv(data_path) 

1. Дослідимо, який філіал супермаркету ('Branch') є найприбутковішим. Для цього знайдіть сумарний прибуток за кожним філіалом і виявіть, який філіал має найвищий.

In [6]:
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [7]:
df.groupby('Branch')['gross income'].sum()

Branch
A    5057.1605
B    5057.0320
C    5265.1765
Name: gross income, dtype: float64

2. В якому місті знайходиться філіал з найвищим прибутком? Може в тому місці нам розмітисти ще один магазин.  
Знайдіть відповідь за допомогою функціоналу Pandas.

In [19]:
df.groupby(['City','Branch'])['gross income'].sum().idxmax()

('Naypyitaw', 'C')

3.1. Створіть зводну таблицю, яка покаже, скільки покупок (інвойсів) було зроблено в кожній з філій (`Branch`) за різними категоріями товарів. Запишіть таблицю в змінну `invoices_by_category` і виведіть змінну на екран.
Ця таблиця допоможе проаналізувати, в якій філії купують найбільше товарів кожної з категорій.

In [22]:
invoices_by_category = pd.pivot_table(df, index='Branch', columns='Product line', values='Invoice ID', aggfunc='count')

invoices_by_category

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Branch,,,,,,
A,60,51,58,47,65,59
B,55,62,50,53,50,62
C,55,65,66,52,45,45


Очікуваний результат:

![](https://drive.google.com/uc?export=view&id=1rueAdko6S3UxIHGtojetTxlES-EyM6Yb)

3.2. Викристовуючи змінну `invoices_by_category` дайте відповідь програмно (тобто значення треба не просто знайти очима, а вивести за допомогою коду), в якому філіалі магазину (`Branch`) найбільше інвойсів із покупкою товарів категорії "Електронні аксесуари" (`Electronic accessories`)?


In [37]:
invoices_by_category['Electronic accessories'].idxmax()

'A'

4-6. **Творче завдання на розвиток аналітичного мислення**

Крок 1. Сформулюйте ТРИ питання (гіпотези) до наявних даних, які допомогли б вам зрозуміти, які користувачі що, де та коли найбільше/найменше купують, аби дати на основі цих гіпотез рекомендації бізнесу. Звісно питання мають бути не тими, на які ми вже відповіли в завданнях модулю.

Крок 2. Знайдіть відповіді на свої питання з допомогою функціоналу pandas.

Крок 3. Напишіть, як відповідь на це питання може бути використана для прийняття бізнес рішень.   
   
 Питання можуть бути будь-якої складності, але їх має бути 3. Кожне питання оцінюється як 1 завдання. Без виконання цього завдання ДЗ не приймається. Якщо є питання щодо виконання - пишіть у чат 🙌


1) __Хто наш клієнт (жінка/ чоловік)? Cкільки клієнтів - чоловіків/жінок по даті (місяць рік)?__

In [74]:
df.Gender.value_counts()

Gender
Female    501
Male      499
Name: count, dtype: int64

In [78]:
df['Date'] = pd.to_datetime(df['Date'])
df['Month_Year'] = df['Date'].dt.to_period('M').astype(str)
gender_by_date = df.groupby(['Month_Year', 'Gender'])['Invoice ID'].count().unstack()
print(gender_by_date)

Gender      Female  Male
Month_Year              
2019-01        176   176
2019-02        164   139
2019-03        161   184


Ця інформація може допомогти обрати маркетингову стратегію та вибір товарів. Якщо частка жінок зростає - розширити асортимент товарів для жінок і навпаки. Зміни по роках покажуть у яку аудиторію варто вкладати маркетинговий бюджет. 

2) __Які товари найбільше/найменьше купують жінки та чоловіки? також в розрізі по містах?__

In [63]:
products_by_gender = pd.pivot_table(df, index=['City','Gender'], columns='Product line', values='Invoice ID', aggfunc='count')

products_by_gender.div(products_by_gender.groupby(level=0).sum()).round(2)

Product line      Electronic accessories  Fashion accessories  \
City      Gender                                                
Mandalay  Female                    0.51                 0.53   
          Male                      0.49                 0.47   
Naypyitaw Female                    0.51                 0.54   
          Male                      0.49                 0.46   
Yangon    Female                    0.47                 0.55   
          Male                      0.53                 0.45   

Product line      Food and beverages  Health and beauty  Home and lifestyle  \
City      Gender                                                              
Mandalay  Female                0.58               0.38                0.44   
          Male                  0.42               0.62                0.56   
Naypyitaw Female                0.58               0.44                0.56   
          Male                  0.42               0.56                0.44   
Yangon    Female                0.40               0.45                0.49   
          Male                  0.60               0.55                0.51   

Product line      Sports and travel  
City      Gender                     
Mandalay  Female               0.48  
          Male                 0.52  
Naypyitaw Female               0.64  
          Male                 0.36  
Yangon    Female               0.49  
          Male                 0.51

Ця інформація дасть розуміння на кого таргетувати конкретні товарні категоріі. Також допоможе оптимізувати асортимент магазинів у кожному місті.

3) __Чи впливає спосіб оплати на середній чек покупця?__

In [84]:
df.groupby('Payment')['Total'].mean().sort_values()

Payment
Ewallet        318.820600
Credit card    324.009878
Cash           326.181890
Name: Total, dtype: float64

Це допоможе вирішити чи потрібно більше POS - терміналів та просувати cashless оплати (кешбек, бонуси) та чи варто робити партнерства з Ewallet сервісами.